In [2]:
from matplotlib import pyplot as plt
from environment_wrapper import *
import networkx as nx
from numpy.random import randint

env = GraphWrapper()
running_reward = 0
for e in range(1):

    env.reset()
    state = env.observe()
    done = False
    episode_reward = 0
    actions = 0
    while not done:
        G, node_inputs, leaf_nodes = state 
        if len(env.leaf_nodes) > 0:
            node = env.leaf_nodes[randint(len(env.leaf_nodes))]
        else :
            node = 0
        state, reward, done = env.step((node, 1), False)
        episode_reward += reward
        actions += 1

    # agent.save(version="2")
    running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward
    print(actions, episode_reward)

    # log results
    if e % 1 == 0:
        print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f} \n'.format(
            e, episode_reward, running_reward))


Using backend: pytorch


KeyboardInterrupt: 

In [3]:
import dgl
import torch
import torch.nn as nn
from dgl.nn.pytorch import GraphConv
import torch.nn.functional as F

class GCN(nn.Module):

    def __init__(self, features=5, hidden_layer_size=5, embedding_size=1):
        super(GCN, self).__init__()
        
        self.conv1 = GraphConv(in_feats=features, out_feats=hidden_layer_size, weight=True, bias=True)
        self.conv1.reset_parameters()
        self.conv2 = GraphConv(in_feats=hidden_layer_size, out_feats=embedding_size, weight=True, bias=True)
        self.conv2.reset_parameters()
        # self.conv3 = GraphConv(hidden_layer_size, embedding_size)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, g, inputs):
        h = inputs
        h = self.conv1(g, h)
        h = torch.sigmoid(h)
        h = self.conv2(g, h)
        h = torch.sigmoid(h)

        return h

gnn = GCN().to("cuda")

g1 = dgl.graph((torch.tensor([0, 1, 2]), torch.tensor([1, 2, 3])))
g2 = dgl.graph((torch.tensor([0, 0, 0, 1]), torch.tensor([0, 1, 2, 0])))
g  = []
size = [4, 3]
g1 = dgl.add_self_loop(g1)
g2 = dgl.add_self_loop(g2)
g.append(g1)
g.append(g2)

leaf_nodes = [[0,3],[2]]



node_inputs = torch.tensor([[ 2, 3, 1, 2 ,3, 4, 5]]).to("cuda")

bg = dgl.batch([g1, g2]).to("cuda")

logits = gnn.forward(bg, node_inputs)

In [1]:
import numpy as np
a = np.array([[1,2,3],[4,5,6]])
np.concatenate((a, np.array([[-1,-1,-1]])), axis=0)

TypeError: only integer scalar arrays can be converted to a scalar index

In [19]:
a[0].add_nodes(1)
a[1].append(5)
a[2].append(3)
a

(Graph(num_nodes=5, num_edges=7,
       ndata_schemes={}
       edata_schemes={}),
 [5],
 [3])